In [ ]:
# Clone the tensorflow models repository from GitHub
!pip uninstall Cython -y # Temporary fix for "No module named 'object_detection'" error
!git clone --depth 1 https://github.com/tensorflow/models

Found existing installation: Cython 3.0.7
Uninstalling Cython-3.0.7:
  Successfully uninstalled Cython-3.0.7
Cloning into 'models'...
remote: Enumerating objects: 4065, done.
remote: Counting objects: 100% (4065/4065), done.
remote: Compressing objects: 100% (3109/3109), done.
remote: Total 4065 (delta 1183), reused 1933 (delta 896), pack-reused 0
Receiving objects: 100% (4065/4065), 56.43 MiB | 10.27 MiB/s, done.
Resolving deltas: 100% (1183/1183), done.
Updating files: 100% (3677/3677), done.


In [ ]:
# Install the Object Detection API (NOTE: This block takes about 10 minutes to finish executing)

# Need to do a temporary fix with PyYAML because Colab isn't able to install PyYAML v5.4.1
!pip install pyyaml==5.3
!pip install /content/models/research/

# Need to downgrade to TF v2.8.0 due to Colab compatibility bug with TF v2.10 (as of 10/03/22)
!pip install tensorflow==2.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.2/268.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.3-cp310-cp310-linux_x86_64.whl size=44244 sha256=294d5866f10c013b218caf0d28fdcc3fa4878112a6c3167f1b3c261f1fe3caf7
  Stored in directory: /root/.cache/pip/wheels/0d/72/68/a263cfc14175636cf26bada99f13b735be1b60a11318e08bfc
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.1
    Uninstalling PyYAML-6.0.1:
      Successfully uninstalled PyYAML-6.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
dask 2023.8.1 requires pyyaml>=5.3.1, but you have pyyaml 5.3 which is incompatible.
distr

In [ ]:
!unzip images.zip
!unzip custom_model_lite_jan_10.zip

Archive:  images.zip
   creating: images/
  inflating: images/.DS_Store        
  inflating: __MACOSX/images/._.DS_Store  
   creating: images/test/
   creating: images/train/
   creating: images/validation/
  inflating: images/test/8496278-grilled-lemon-pepper-chicken-mfs-beauty-1-2x3-d13c6ac20c8d4b9dbef84f14203e2876.JPG  
  inflating: __MACOSX/images/test/._8496278-grilled-lemon-pepper-chicken-mfs-beauty-1-2x3-d13c6ac20c8d4b9dbef84f14203e2876.JPG  
  inflating: images/test/8496278-grilled-lemon-pepper-chicken-mfs-beauty-1-2x3-d13c6ac20c8d4b9dbef84f14203e2876.xml  
  inflating: __MACOSX/images/test/._8496278-grilled-lemon-pepper-chicken-mfs-beauty-1-2x3-d13c6ac20c8d4b9dbef84f14203e2876.xml  
  inflating: images/test/941f50-splendid-table-french-fries.xml  
  inflating: __MACOSX/images/test/._941f50-splendid-table-french-fries.xml  
  inflating: images/test/941f50-splendid-table-french-fries.JPG  
  inflating: __MACOSX/images/test/._941f50-splendid-table-french-fries.JPG  
  inflating:

In [ ]:
import glob
import tensorflow as tf
# Get list of all images in train directory
image_path = '/content/images/train'

jpg_file_list = glob.glob(image_path + '/*.jpg')
JPG_file_list = glob.glob(image_path + '/*.JPG')
png_file_list = glob.glob(image_path + '/*.png')
bmp_file_list = glob.glob(image_path + '/*.bmp')

quant_image_list = jpg_file_list + JPG_file_list + png_file_list + bmp_file_list

In [ ]:
# A generator that provides a representative dataset
# Code modified from https://colab.research.google.com/github/google-coral/tutorials/blob/master/retrain_classification_ptq_tf2.ipynb

# First, get input details for model so we know how to preprocess images

from tensorflow.lite.python.interpreter import Interpreter

PATH_TO_MODEL='/content/custom_model_lite/detect.tflite'   # Path to .tflite model file
interpreter = Interpreter(model_path=PATH_TO_MODEL)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

import random

def representative_data_gen():
  dataset_list = quant_image_list
  quant_num = 300
  for i in range(quant_num):
    pick_me = random.choice(dataset_list)
    image = tf.io.read_file(pick_me)

    if pick_me.endswith('.jpg') or pick_me.endswith('.JPG'):
      image = tf.io.decode_jpeg(image, channels=3)
    elif pick_me.endswith('.png'):
      image = tf.io.decode_png(image, channels=3)
    elif pick_me.endswith('.bmp'):
      image = tf.io.decode_bmp(image, channels=3)

    image = tf.image.resize(image, [width, height])  # TO DO: Replace 300s with an automatic way of reading network input size
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

In [ ]:
# Initialize converter module
converter = tf.lite.TFLiteConverter.from_saved_model('/content/custom_model_lite/saved_model')

# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen
# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
converter.target_spec.supported_types = [tf.int8]
# These set the input tensors to uint8 and output tensors to float32
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.float32
tflite_model = converter.convert()

with open('/content/detect_quant.tflite', 'wb') as f:
  f.write(tflite_model)